In [1]:
import os
import scipy.misc
import numpy as np
import tensorflow as tf
from DCGAN.model import DCGAN
from DCGAN.utils import pp, visualize, show_all_variables

In [2]:
flags = tf.app.flags
flags.DEFINE_integer("epoch", 20, "Epoch to train [25]")
flags.DEFINE_float("learning_rate", 0.0001, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.1, "Momentum term of adam [0.5]")
flags.DEFINE_float("train_size", np.inf, "The size of train images [np.inf]")
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_integer("input_height", 218, "The size of image to use (will be center cropped). [108]")
flags.DEFINE_integer("input_width", 178, "The size of image to use (will be center cropped). If None, same value as input_height [None]")
flags.DEFINE_integer("output_height", 128, "The size of the output images to produce [64]")
flags.DEFINE_integer("output_width", 128, "The size of the output images to produce. If None, same value as output_height [None]")
flags.DEFINE_string("dataset", "celebA", "The name of data directory.")
flags.DEFINE_string("input_fname_pattern", "*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("data_dir", "./datasets", "Root directory of dataset [data]")
flags.DEFINE_string("sample_dir", "./samples", "Directory name to save the image samples [samples]")
flags.DEFINE_boolean("train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("crop", True, "whether using crop [False]")
flags.DEFINE_integer("z_dims", 128, "dims of z. [100]")
flags.DEFINE_integer("sample_num",64,"number of samples.")
FLAGS = flags.FLAGS

In [3]:
def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if FLAGS.input_width is None:
        FLAGS.input_width = FLAGS.input_height
    if FLAGS.output_width is None:
        FLAGS.output_width = FLAGS.output_height

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)

  #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
    run_config = tf.ConfigProto()
    run_config.gpu_options.allow_growth=True
    
    tf.reset_default_graph()
    with tf.Session(config=run_config) as sess:
        dcgan = DCGAN(
              sess,
              input_width=FLAGS.input_width,
              input_height=FLAGS.input_height,
              output_width=FLAGS.output_width,
              output_height=FLAGS.output_height,
              batch_size=FLAGS.batch_size,
              sample_num=FLAGS.sample_num,
              z_dim=FLAGS.z_dims,
              dataset_name=FLAGS.dataset,
              input_fname_pattern=FLAGS.input_fname_pattern,
              crop=FLAGS.crop,
              gen_first_dim=128,
              dis_first_dim=128,
              checkpoint_dir=FLAGS.checkpoint_dir,
              data_dir=FLAGS.data_dir)
        print("show_all_variables ====>")
        show_all_variables()

        if FLAGS.train:
            dcgan.train(FLAGS)
        else:
            if not dcgan.load(FLAGS.checkpoint_dir)[0]:
                raise Exception("[!] Train a model first, then run test mode")


        # Below is codes for visualization
        OPTION = 1
        visualize(sess, dcgan, FLAGS, OPTION)

In [ ]:
'''
{'batch_size': <absl.flags._flag.Flag object at 0x0000023BD3852208>,
 'beta1': <absl.flags._flag.Flag object at 0x0000023BD38520F0>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x0000023BD38526A0>,
 'crop': <absl.flags._flag.BooleanFlag object at 0x0000023BD3852940>,
 'data_dir': <absl.flags._flag.Flag object at 0x0000023BD3852780>,
 'dataset': <absl.flags._flag.Flag object at 0x0000023BD3852550>,
 'epoch': <absl.flags._flag.Flag object at 0x0000023BD38520B8>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x0000023BD3862240>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x0000023BD3862240>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x0000023BD3862320>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x0000023BD3862390>,
 'input_fname_pattern': <absl.flags._flag.Flag object at 0x0000023BD38525F8>,
 'input_height': <absl.flags._flag.Flag object at 0x0000023BD38522E8>,
 'input_width': <absl.flags._flag.Flag object at 0x0000023BD3852358>,
 'learning_rate': <absl.flags._flag.Flag object at 0x0000023BD3852160>,
 'output_height': <absl.flags._flag.Flag object at 0x0000023BD3852400>,
 'output_width': <absl.flags._flag.Flag object at 0x0000023BD38524A8>,
 'sample_dir': <absl.flags._flag.Flag object at 0x0000023BD38527B8>,
 'sample_num': <absl.flags._flag.Flag object at 0x0000023BD38529B0>,
 'train': <absl.flags._flag.BooleanFlag object at 0x0000023BD38528D0>,
 'train_size': <absl.flags._flag.Flag object at 0x0000023BD38521D0>,
 'z_dims': <absl.flags._flag.Flag object at 0x0000023BD3852978>}
WARNING:tensorflow:From F:\Anaconda\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
show_all_variables ====>
---------
Variables: name (type shape) [size]
---------
generator/g_h0_lin/Matrix:0 (float32_ref 128x65536) [8388608, bytes: 33554432]
generator/g_h0_lin/bias:0 (float32_ref 65536) [65536, bytes: 262144]
generator/g_bn0/beta:0 (float32_ref 1024) [1024, bytes: 4096]
generator/g_bn0/gamma:0 (float32_ref 1024) [1024, bytes: 4096]
generator/g_h1/w:0 (float32_ref 5x5x512x1024) [13107200, bytes: 52428800]
generator/g_h1/biases:0 (float32_ref 512) [512, bytes: 2048]
generator/g_bn1/beta:0 (float32_ref 512) [512, bytes: 2048]
generator/g_bn1/gamma:0 (float32_ref 512) [512, bytes: 2048]
generator/g_h2/w:0 (float32_ref 5x5x256x512) [3276800, bytes: 13107200]
generator/g_h2/biases:0 (float32_ref 256) [256, bytes: 1024]
generator/g_bn2/beta:0 (float32_ref 256) [256, bytes: 1024]
generator/g_bn2/gamma:0 (float32_ref 256) [256, bytes: 1024]
generator/g_h3/w:0 (float32_ref 5x5x128x256) [819200, bytes: 3276800]
generator/g_h3/biases:0 (float32_ref 128) [128, bytes: 512]
generator/g_bn3/beta:0 (float32_ref 128) [128, bytes: 512]
generator/g_bn3/gamma:0 (float32_ref 128) [128, bytes: 512]
generator/g_h4/w:0 (float32_ref 5x5x3x128) [9600, bytes: 38400]
generator/g_h4/biases:0 (float32_ref 3) [3, bytes: 12]
discriminator/d_h0_conv/w:0 (float32_ref 5x5x3x128) [9600, bytes: 38400]
discriminator/d_h0_conv/biases:0 (float32_ref 128) [128, bytes: 512]
discriminator/d_h1_conv/w:0 (float32_ref 5x5x128x256) [819200, bytes: 3276800]
discriminator/d_h1_conv/biases:0 (float32_ref 256) [256, bytes: 1024]
discriminator/d_bn1/beta:0 (float32_ref 256) [256, bytes: 1024]
discriminator/d_bn1/gamma:0 (float32_ref 256) [256, bytes: 1024]
discriminator/d_h2_conv/w:0 (float32_ref 5x5x256x512) [3276800, bytes: 13107200]
discriminator/d_h2_conv/biases:0 (float32_ref 512) [512, bytes: 2048]
discriminator/d_bn2/beta:0 (float32_ref 512) [512, bytes: 2048]
discriminator/d_bn2/gamma:0 (float32_ref 512) [512, bytes: 2048]
discriminator/d_h3_conv/w:0 (float32_ref 5x5x512x1024) [13107200, bytes: 52428800]
discriminator/d_h3_conv/biases:0 (float32_ref 1024) [1024, bytes: 4096]
discriminator/d_bn3/beta:0 (float32_ref 1024) [1024, bytes: 4096]
discriminator/d_bn3/gamma:0 (float32_ref 1024) [1024, bytes: 4096]
discriminator/d_h4_lin/Matrix:0 (float32_ref 65536x1) [65536, bytes: 262144]
discriminator/d_h4_lin/bias:0 (float32_ref 1) [1, bytes: 4]
Total size of variables: 42955524
Total bytes of variables: 171822096
 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0/20] [  99/3165] time: 364.8617, d_loss: 4.95020962, g_loss: 5.0778040927
one pic error!...
Epoch: [ 0/20] [ 199/3165] time: 711.8495, d_loss: 3.89164758, g_loss: 0.093089986
one pic error!...
Epoch: [ 0/20] [ 299/3165] time: 1058.7980, d_loss: 2.57458067, g_loss: 0.16823277
one pic error!...
Epoch: [ 0/20] [ 399/3165] time: 1406.1047, d_loss: 3.52352118, g_loss: 0.08555351
one pic error!...
Epoch: [ 0/20] [ 499/3165] time: 1753.5876, d_loss: 1.18351293, g_loss: 1.48786438
one pic error!...
Epoch: [ 0/20] [ 599/3165] time: 2103.2846, d_loss: 1.74076867, g_loss: 4.38535023
one pic error!...
Epoch: [ 0/20] [ 699/3165] time: 2450.2440, d_loss: 1.06066048, g_loss: 1.57614088
one pic error!...
Epoch: [ 0/20] [ 799/3165] time: 2797.8760, d_loss: 1.61192703, g_loss: 2.87611771
one pic error!...
Epoch: [ 0/20] [ 899/3165] time: 3145.0294, d_loss: 1.01890910, g_loss: 1.68074191
one pic error!...
Epoch: [ 0/20] [ 999/3165] time: 3492.1241, d_loss: 0.83397329, g_loss: 1.64063883
one pic error!...
Epoch: [ 0/20] [1099/3165] time: 3841.2700, d_loss: 4.36886835, g_loss: 0.02226834
one pic error!...
Epoch: [ 0/20] [1199/3165] time: 4188.6391, d_loss: 1.71518874, g_loss: 0.32852918
one pic error!...
Epoch: [ 0/20] [1299/3165] time: 4535.7589, d_loss: 1.61059260, g_loss: 2.10193586
one pic error!...
Epoch: [ 0/20] [1399/3165] time: 4883.3970, d_loss: 1.60312831, g_loss: 0.39874279
one pic error!...
Epoch: [ 0/20] [1499/3165] time: 5230.7739, d_loss: 1.11917043, g_loss: 0.82740176
one pic error!...
Epoch: [ 0/20] [1599/3165] time: 5579.9783, d_loss: 2.45716238, g_loss: 0.14152119
one pic error!...
Epoch: [ 0/20] [1699/3165] time: 5927.3614, d_loss: 3.48507476, g_loss: 0.04961297
one pic error!...
Epoch: [ 0/20] [1799/3165] time: 6274.4292, d_loss: 1.09856975, g_loss: 1.57889628
one pic error!...
Epoch: [ 0/20] [1899/3165] time: 6622.1088, d_loss: 3.25979710, g_loss: 0.06313445
one pic error!...
Epoch: [ 0/20] [1999/3165] time: 6969.9929, d_loss: 1.19863081, g_loss: 1.06598759
one pic error!...
Epoch: [ 0/20] [2099/3165] time: 7320.1014, d_loss: 1.44743574, g_loss: 2.15842366
one pic error!...
Epoch: [ 0/20] [2199/3165] time: 7667.4475, d_loss: 1.30056310, g_loss: 1.75889230
one pic error!...
Epoch: [ 0/20] [2299/3165] time: 8015.0562, d_loss: 1.91722667, g_loss: 0.19059776
one pic error!...
Epoch: [ 0/20] [2399/3165] time: 8362.7751, d_loss: 1.90751171, g_loss: 0.23814550
one pic error!...
Epoch: [ 0/20] [2499/3165] time: 8710.8736, d_loss: 2.21131301, g_loss: 0.14602351
one pic error!...
Epoch: [ 0/20] [2500/3165] time: 8714.3193, d_loss: 2.06434131, g_loss: 2.47171021WARNING:tensorflow:From F:\Anaconda\lib\site-packages\tensorflow\python\training\saver.py:966: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: [ 0/20] [2599/3165] time: 9059.8696, d_loss: 1.63977718, g_loss: 0.33740509
one pic error!...
Epoch: [ 0/20] [2699/3165] time: 9407.7090, d_loss: 1.42700362, g_loss: 1.40333688
one pic error!...
Epoch: [ 0/20] [2799/3165] time: 9755.2873, d_loss: 1.47228134, g_loss: 1.35440958
one pic error!...
Epoch: [ 0/20] [2899/3165] time: 10103.0748, d_loss: 2.00104022, g_loss: 0.28197116
one pic error!...
Epoch: [ 0/20] [2999/3165] time: 10450.4289, d_loss: 1.57471383, g_loss: 0.51220393
one pic error!...
Epoch: [ 0/20] [3099/3165] time: 10799.8732, d_loss: 1.42515361, g_loss: 0.72915757
one pic error!...
Epoch: [ 1/20] [  34/3165] time: 11130.2174, d_loss: 1.33131969, g_loss: 1.22643018
one pic error!...
Epoch: [ 1/20] [ 134/3165] time: 11426.2962, d_loss: 2.20984697, g_loss: 0.16169474
one pic error!...
Epoch: [ 1/20] [ 234/3165] time: 11722.5119, d_loss: 1.85299277, g_loss: 0.32054746
one pic error!...
Epoch: [ 1/20] [ 334/3165] time: 12018.7482, d_loss: 1.94912410, g_loss: 0.18118982
one pic error!...
Epoch: [ 1/20] [ 434/3165] time: 12316.9395, d_loss: 1.14488804, g_loss: 1.98734117
one pic error!...
Epoch: [ 1/20] [ 534/3165] time: 12613.3882, d_loss: 2.09966969, g_loss: 0.16799971
one pic error!...
Epoch: [ 1/20] [ 634/3165] time: 12909.8815, d_loss: 1.09448552, g_loss: 1.06727052
one pic error!...
Epoch: [ 1/20] [ 734/3165] time: 13206.3351, d_loss: 1.53715301, g_loss: 0.33655715
one pic error!...
Epoch: [ 1/20] [ 834/3165] time: 13502.7975, d_loss: 1.83097672, g_loss: 0.52517283
one pic error!...
Epoch: [ 1/20] [ 934/3165] time: 13801.5281, d_loss: 2.19901443, g_loss: 0.16413057
one pic error!...
Epoch: [ 1/20] [1034/3165] time: 14098.2505, d_loss: 4.84696627, g_loss: 0.01719577
one pic error!...
Epoch: [ 1/20] [1134/3165] time: 14394.8947, d_loss: 2.46671367, g_loss: 2.14445591
one pic error!...
Epoch: [ 1/20] [1234/3165] time: 14691.7600, d_loss: 2.26268840, g_loss: 0.16362417
one pic error!...
Epoch: [ 1/20] [1334/3165] time: 14988.4314, d_loss: 2.77741528, g_loss: 0.08780229
one pic error!...
Epoch: [ 1/20] [1434/3165] time: 15287.0839, d_loss: 1.50423789, g_loss: 1.52901518
one pic error!...
Epoch: [ 1/20] [1534/3165] time: 15583.7969, d_loss: 2.59772849, g_loss: 3.52758718
one pic error!...
Epoch: [ 1/20] [1634/3165] time: 15880.6886, d_loss: 3.08515716, g_loss: 0.06266402
one pic error!...
Epoch: [ 1/20] [1734/3165] time: 16177.4585, d_loss: 3.62904191, g_loss: 0.03532697
one pic error!...
Epoch: [ 1/20] [1834/3165] time: 16474.2487, d_loss: 1.59994984, g_loss: 0.33224231
one pic error!...
Epoch: [ 1/20] [1934/3165] time: 16772.8585, d_loss: 2.16095924, g_loss: 2.49732542
one pic error!...
Epoch: [ 1/20] [2034/3165] time: 17069.5755, d_loss: 0.88382316, g_loss: 1.24618936
one pic error!...
Epoch: [ 1/20] [2134/3165] time: 17366.3266, d_loss: 1.67071688, g_loss: 0.34508246
one pic error!...
Epoch: [ 1/20] [2234/3165] time: 17663.1898, d_loss: 1.93981206, g_loss: 0.24937952
one pic error!...
Epoch: [ 1/20] [2334/3165] time: 17959.8894, d_loss: 1.24118030, g_loss: 0.73872536
one pic error!...
Epoch: [ 1/20] [2434/3165] time: 18258.5406, d_loss: 2.34301543, g_loss: 0.15479717
one pic error!...
Epoch: [ 1/20] [2534/3165] time: 18555.1932, d_loss: 1.33615458, g_loss: 0.86621046
one pic error!...
Epoch: [ 1/20] [2634/3165] time: 18851.8839, d_loss: 1.35993361, g_loss: 0.93661988
one pic error!...
Epoch: [ 1/20] [2734/3165] time: 19148.6238, d_loss: 1.37006724, g_loss: 1.74394751
one pic error!...
Epoch: [ 1/20] [2834/3165] time: 19445.6070, d_loss: 1.64392483, g_loss: 0.33781850
one pic error!...
Epoch: [ 1/20] [2934/3165] time: 19744.1962, d_loss: 1.73552990, g_loss: 0.25684130
one pic error!...
Epoch: [ 1/20] [3034/3165] time: 20041.0523, d_loss: 2.73934531, g_loss: 0.08660771
one pic error!...
Epoch: [ 1/20] [3134/3165] time: 20337.6147, d_loss: 1.86802673, g_loss: 0.24587944
one pic error!...
Epoch: [ 2/20] [  69/3165] time: 20635.1991, d_loss: 2.77548766, g_loss: 0.08076704
one pic error!...
Epoch: [ 2/20] [ 169/3165] time: 20931.9046, d_loss: 1.88940835, g_loss: 0.23752041
one pic error!...
Epoch: [ 2/20] [ 269/3165] time: 21230.3458, d_loss: 0.91674650, g_loss: 1.35623062
one pic error!...
Epoch: [ 2/20] [ 369/3165] time: 21527.2685, d_loss: 3.22984862, g_loss: 0.07159962
one pic error!...
Epoch: [ 2/20] [ 469/3165] time: 21824.0142, d_loss: 2.06553316, g_loss: 0.18839520
one pic error!...
Epoch: [ 2/20] [ 569/3165] time: 22120.8334, d_loss: 1.69562089, g_loss: 0.28953499
one pic error!...
Epoch: [ 2/20] [ 669/3165] time: 22417.6393, d_loss: 1.17512822, g_loss: 0.86863881
one pic error!...
Epoch: [ 2/20] [ 769/3165] time: 22716.1519, d_loss: 1.18201065, g_loss: 1.31847858
one pic error!...
Epoch: [ 2/20] [ 869/3165] time: 23012.9637, d_loss: 1.97725844, g_loss: 0.22884095
one pic error!...
Epoch: [ 2/20] [ 969/3165] time: 23309.6995, d_loss: 3.63531709, g_loss: 0.04628861
one pic error!...
Epoch: [ 2/20] [1069/3165] time: 23606.5087, d_loss: 1.31466651, g_loss: 1.84168935
one pic error!...
Epoch: [ 2/20] [1169/3165] time: 23903.3428, d_loss: 2.12977028, g_loss: 0.15622897
one pic error!...
Epoch: [ 2/20] [1269/3165] time: 24201.7447, d_loss: 0.94352007, g_loss: 0.98805332
one pic error!...
Epoch: [ 2/20] [1369/3165] time: 24498.6924, d_loss: 3.11692333, g_loss: 0.06195745
one pic error!...
Epoch: [ 2/20] [1469/3165] time: 24795.5274, d_loss: 1.66505170, g_loss: 0.28456756
one pic error!...
Epoch: [ 2/20] [1569/3165] time: 25092.3413, d_loss: 1.42560124, g_loss: 1.27694666
one pic error!...
Epoch: [ 2/20] [1669/3165] time: 25389.0791, d_loss: 0.97189939, g_loss: 0.74478650
one pic error!...
Epoch: [ 2/20] [1769/3165] time: 25687.5659, d_loss: 0.98253626, g_loss: 1.36013258
one pic error!...
Epoch: [ 2/20] [1869/3165] time: 25984.3352, d_loss: 2.01105666, g_loss: 0.17037843
one pic error!...
Epoch: [ 2/20] [1969/3165] time: 26281.0211, d_loss: 1.97940660, g_loss: 0.18845555
one pic error!...
Epoch: [ 2/20] [2069/3165] time: 26577.6766, d_loss: 1.14566350, g_loss: 2.11303163
one pic error!...
Epoch: [ 2/20] [2169/3165] time: 26874.5502, d_loss: 1.76611531, g_loss: 0.25523490
one pic error!...
Epoch: [ 2/20] [2269/3165] time: 27172.8889, d_loss: 0.96706784, g_loss: 0.81465560
one pic error!...
Epoch: [ 2/20] [2369/3165] time: 27469.7967, d_loss: 1.29067278, g_loss: 0.87713838
one pic error!...
Epoch: [ 2/20] [2469/3165] time: 27766.4995, d_loss: 4.52937651, g_loss: 0.01755633
one pic error!...
Epoch: [ 2/20] [2569/3165] time: 28063.4299, d_loss: 4.00037384, g_loss: 0.02947185
one pic error!...
Epoch: [ 2/20] [2669/3165] time: 28360.3467, d_loss: 1.54769707, g_loss: 1.69941866
one pic error!...
Epoch: [ 2/20] [2769/3165] time: 28658.9727, d_loss: 1.26670623, g_loss: 1.04667330
one pic error!...
Epoch: [ 2/20] [2869/3165] time: 28955.8299, d_loss: 0.86700046, g_loss: 1.69224739
one pic error!...
Epoch: [ 2/20] [2969/3165] time: 29252.6086, d_loss: 1.04057062, g_loss: 0.74678600
one pic error!...
Epoch: [ 2/20] [3069/3165] time: 29549.4762, d_loss: 1.08920431, g_loss: 0.60007107
one pic error!...
Epoch: [ 3/20] [   4/3165] time: 29847.3122, d_loss: 1.02080059, g_loss: 1.42818916
one pic error!...
Epoch: [ 3/20] [ 104/3165] time: 30145.9976, d_loss: 1.06071639, g_loss: 0.94285232
one pic error!...
Epoch: [ 3/20] [ 204/3165] time: 30442.7413, d_loss: 1.24137473, g_loss: 0.70869666
one pic error!...
Epoch: [ 3/20] [ 304/3165] time: 30739.5352, d_loss: 0.74326336, g_loss: 1.38822114
one pic error!...
Epoch: [ 3/20] [ 404/3165] time: 31036.3233, d_loss: 1.99417770, g_loss: 0.22566757
one pic error!...
Epoch: [ 3/20] [ 504/3165] time: 31333.1679, d_loss: 1.62450838, g_loss: 0.33134845
one pic error!...
Epoch: [ 3/20] [ 604/3165] time: 31639.7462, d_loss: 1.87204647, g_loss: 0.20469528
one pic error!...
Epoch: [ 3/20] [ 674/3165] time: 31849.5699, d_loss: 1.40353346, g_loss: 1.38477314
'''

In [4]:
if __name__ == '__main__':
    tf.app.run()

{'batch_size': <absl.flags._flag.Flag object at 0x0000027D67A6F160>,
 'beta1': <absl.flags._flag.Flag object at 0x0000027D67A6F048>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x0000027D67A6F6D8>,
 'crop': <absl.flags._flag.BooleanFlag object at 0x0000027D67A6F940>,
 'data_dir': <absl.flags._flag.Flag object at 0x0000027D67A6F780>,
 'dataset': <absl.flags._flag.Flag object at 0x0000027D67A6F6A0>,
 'epoch': <absl.flags._flag.Flag object at 0x0000027D67A66FD0>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x0000027D573F23C8>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x0000027D573F23C8>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x0000027D67A57A58>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x0000027D67A57B00>,
 'input_fname_pattern': <absl.flags._flag.Flag object at 0x0000027D67A6F550>,
 'input_height': <absl.flags._flag.Flag object at 0x0000027D67A6F240>,
 'input_width': <absl.flags._flag.F

Epoch: [ 0/20] [2698/3165] time: 8207.4864, d_loss: 0.73608792, g_loss: 1.31274354
[Sample] d_loss: 0.42260620, g_loss: 4.58591032
Epoch: [ 0/20] [2798/3165] time: 8514.8519, d_loss: 0.76234078, g_loss: 1.31840241
[Sample] d_loss: 0.48858327, g_loss: 3.69824028
Epoch: [ 0/20] [2898/3165] time: 8822.2272, d_loss: 2.84743547, g_loss: 0.08359335
[Sample] d_loss: 0.87696499, g_loss: 0.82111216
Epoch: [ 0/20] [2998/3165] time: 9129.6557, d_loss: 0.44457582, g_loss: 1.58229518
[Sample] d_loss: 0.24720268, g_loss: 5.44150400
Epoch: [ 0/20] [2999/3165] time: 9133.5948, d_loss: 2.54664850, g_loss: 0.11017883WARNING:tensorflow:From F:\Anaconda\lib\site-packages\tensorflow\python\training\saver.py:966: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: [ 0/20] [3098/3165] time: 9439.2284, d_loss: 2.72630978, g_loss: 0.08888321
[Sa

KeyboardInterrupt: 

In [4]:
if __name__ == '__main__':
    tf.app.run()

{'batch_size': <absl.flags._flag.Flag object at 0x000001E4610A0208>,
 'beta1': <absl.flags._flag.Flag object at 0x000001E4610A00F0>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x000001E4610A0828>,
 'crop': <absl.flags._flag.BooleanFlag object at 0x000001E4610A0A90>,
 'data_dir': <absl.flags._flag.Flag object at 0x000001E4610A08D0>,
 'dataset': <absl.flags._flag.Flag object at 0x000001E4610A0550>,
 'epoch': <absl.flags._flag.Flag object at 0x000001E4610A00B8>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x000001E461088898>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x000001E461088898>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x000001E45F753DD8>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x000001E461088B00>,
 'input_fname_pattern': <absl.flags._flag.Flag object at 0x000001E4610A05F8>,
 'input_height': <absl.flags._flag.Flag object at 0x000001E4610A02E8>,
 'input_width': <absl.flags._flag.F

Epoch: [ 0/20] [2698/3165] time: 9496.0205, d_loss: 0.17307246, g_loss: 4.1237659515
[Sample] d_loss: 0.12987903, g_loss: 5.06993151
Epoch: [ 0/20] [2798/3165] time: 9854.9646, d_loss: 0.20826751, g_loss: 2.798265931
[Sample] d_loss: 0.24223019, g_loss: 5.26784992
Epoch: [ 0/20] [2898/3165] time: 10213.7991, d_loss: 1.17012942, g_loss: 0.66056120
[Sample] d_loss: 0.23621772, g_loss: 3.13698626
Epoch: [ 0/20] [2998/3165] time: 10572.7053, d_loss: 0.70428067, g_loss: 1.19905281
[Sample] d_loss: 0.20160842, g_loss: 5.46948433
Epoch: [ 0/20] [2999/3165] time: 10577.1504, d_loss: 2.34315777, g_loss: 0.20407984WARNING:tensorflow:From F:\Anaconda\lib\site-packages\tensorflow\python\training\saver.py:966: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: [ 0/20] [3098/3165] time: 10933.6392, d_loss: 1.65627944, g_loss: 0.34109

KeyboardInterrupt: 